In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from nltk.tokenize import RegexpTokenizer

In [151]:
coefs = pd.read_csv('./coef_ranks.csv')

In [152]:
coefs = coefs.rename(columns={
    'Unnamed: 0':'gram',
    '0':'coeff'
})

In [153]:
#First I assign the full text string I want to map to a variable, after cleaning it by inserting \ before apostrophes
comment = 'You need to be able to distinguish between a girl that likes the attention youre giving her from a girl that likes you. Most girls at a party will smile and talk to you if you approach them with confidence. After you talk to her though, you need to find a way for her to invest in the interaction. If she never invests in it you never really know what she wants. Walking away from the girl without getting any info is a good way to do this. You started the first conversation, now shes on the spot. If she doesnt come up to you and talk to you again, she isnt getting your number. This forces her to be more than just a receiver of attention. If youre leaving the party, or think you might not see her again at that party. Give her your contact info, dont take any of hers. Shes then forced to reach out to you if she wants things to go anywhere. Not only will this tell you if shes actually interested in you, it will make her more attracted. She will see that you arent desperate, and that makes you more attractive. Girls can sense when the only reason youre talking to them is to get their number or something like that. They can see right through you. When you walk away without the number, she realizes youre not just trying to get her number, you actually just wanted to talk to her. That is attractive'


In [154]:
#Here I convert it to a list of lowercase tokens
tokenizer = RegexpTokenizer(r'\w+')
comment_tokens = tokenizer.tokenize(comment.lower())

In [155]:
#Here I reduce the coefficient matrix to only those grams in the comment text and their coefficients
coefs['mask'] = coefs['gram'].map(lambda x: x if x in comment_tokens else np.nan)
commentdf = coefs.dropna()
commentdf.drop('mask',axis=1)

,gram,coeff
4,girls,-8.262061
6,girl,-7.893103
8,her,-6.864349
9,you,-6.782092
12,she,-6.401139
72,your,-3.670169
85,dont,-3.526464
91,invest,-3.435114
122,attractive,-3.173911
134,then,-3.126585


In [156]:
commentdf = commentdf.drop('mask',axis=1)

In [157]:
commentdf.to_csv('commentdf.csv')

### Text Heatmap in Tableau

I'm going to import the commentdf csv into Tableau and attempt to present it as an ordered text field, in the same order as the original comment, and then essentially heatmap it for association with subreddit over the various grams, ideally with a color divergent blue red map with gray in the middle to represent weak to no association with one or the other.  We'll see how this works.

First I need to fix the order problem.

In [159]:
commentdf['indexincomment'] = commentdf['gram'].map(lambda x: comment.find(x))

In [165]:
tableau = commentdf.sort_values('indexincomment').reset_index()

In [166]:
tableau.to_csv('tableau.csv')

<img src="./images/text-heatmap.png" />

The key to executing this was to use the gram column as text dimension, and the coefficient column as a color measure.  Then to set a custom divergent color map with the desired colors, and make sure the center is set at 0, and the start and end are set at equal magnitude opposite sign, so the colors will reflect the degree of severity to which coefficients are higher magnitude to one side or the other, if they are, which they were in this case.  

Stop words had been withheld in this model, and this also doesn't show words that were repeated in the comment text.  But even without the repeats of 'girls' for example, you can see clearly that the passage leans broadly toward r/AskMen.